In [1]:
# Load the Pandas libraries with alias 'pd' 
import pandas as pd 
import numpy as np
from keras.models import Sequential
from keras.layers import Activation, Dropout
from keras.optimizers import SGD
from keras.constraints import max_norm
from sklearn.linear_model import LogisticRegression
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import regularizers
from keras.layers import Dense
from sklearn import preprocessing
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn import model_selection
from sklearn.externals import joblib

# Read data
#EI
train_data_EI = pd.read_csv("../data/traintest/mbti_EI_train")
train_data_EI_target = pd.read_csv("../data/traintest/mbti_EI_train_target", header=None, usecols=[1])
test_data_EI = pd.read_csv("../data/traintest/mbti_EI_test")
test_data_EI_target = pd.read_csv("../data/traintest/mbti_EI_test_target", header=None, usecols=[1])
#FT
train_data_FT = pd.read_csv("../data/traintest/mbti_FT_train")
train_data_FT_target = pd.read_csv("../data/traintest/mbti_FT_train_target", header=None, usecols=[1])
test_data_FT = pd.read_csv("../data/traintest/mbti_FT_test")
test_data_FT_target = pd.read_csv("../data/traintest/mbti_FT_test_target", header=None, usecols=[1])
#NS
train_data_NS = pd.read_csv("../data/traintest/mbti_NS_train")
train_data_NS_target = pd.read_csv("../data/traintest/mbti_NS_train_target", header=None, usecols=[1])
test_data_NS = pd.read_csv("../data/traintest/mbti_NS_test")
test_data_NS_target = pd.read_csv("../data/traintest/mbti_NS_test_target", header=None, usecols=[1])
#PJ
train_data_PJ = pd.read_csv("../data/traintest/mbti_PJ_train")
train_data_PJ_target = pd.read_csv("../data/traintest/mbti_PJ_train_target", header=None, usecols=[1])
test_data_PJ = pd.read_csv("../data/traintest/mbti_PJ_test")
test_data_PJ_target = pd.read_csv("../data/traintest/mbti_PJ_test_target", header=None, usecols=[1])
#

Using TensorFlow backend.


In [2]:

train_data_EI.head() 
train_data_EI_target.columns


Int64Index([1], dtype='int64')

In [3]:
#covert targets to binary 

#training data manipulation
train_data_EI_target[1] = train_data_EI_target[1].map({'I': '1', 'E': '0'}) #change Intravert'I'/Extrovert'E' to 1 and 0 for logistical regression
train_data_FT_target[1] = train_data_FT_target[1].map({'T': '1', 'F': '0'}) #change T/F to 1 and 0 for logistical regression
train_data_NS_target[1] = train_data_NS_target[1].map({'N': '1', 'S': '0'}) #change N/S to 1 and 0 for logistical regression
train_data_PJ_target[1] = train_data_PJ_target[1].map({'P': '1', 'J': '0'}) #change P/J to 1 and 0 for logistical regression
train_data_EI_target.head()

,1
0,1
1,0
2,1
3,1
4,0


In [4]:
#test data manipulation
test_data_EI_target[1] = test_data_EI_target[1].map({'I': '1', 'E': '0'}) #change Intravert'I'/Extrovert'E' to 1 and 0 for logistical regression
test_data_FT_target[1] = test_data_FT_target[1].map({'T': '1', 'F': '0'}) #change T/F to 1 and 0 for logistical regression
test_data_NS_target[1] = test_data_NS_target[1].map({'N': '1', 'S': '0'}) #change N/S to 1 and 0 for logistical regression
test_data_PJ_target[1] = test_data_PJ_target[1].map({'P': '1', 'J': '0'}) #change P/J to 1 and 0 for logistical regression
test_data_FT_target.head()

,1
0,0
1,1
2,1
3,1
4,1


In [5]:
#scale data
#EI
train_EI_scaler = preprocessing.StandardScaler().fit(train_data_EI)
train_EI_scaled = train_EI_scaler.transform(train_data_EI)
test_EI_scaled = train_EI_scaler.transform(test_data_EI)
#FT
train_FT_scaler = preprocessing.StandardScaler().fit(train_data_FT)
train_FT_scaled = train_EI_scaler.transform(train_data_FT)
test_FT_scaled = train_EI_scaler.transform(test_data_FT)
#NS
train_NS_scaler = preprocessing.StandardScaler().fit(train_data_NS)
train_NS_scaled = train_EI_scaler.transform(train_data_NS)
test_NS_scaled = train_EI_scaler.transform(test_data_NS)
#PJ
train_PJ_scaler = preprocessing.StandardScaler().fit(train_data_PJ)
train_PJ_scaled = train_EI_scaler.transform(train_data_PJ)
test_PJ_scaled = train_EI_scaler.transform(test_data_PJ)

E:\Anaconda\lib\site-packages\sklearn\preprocessing\data.py:617: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
E:\Anaconda\lib\site-packages\ipykernel_launcher.py:4: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  after removing the cwd from sys.path.
E:\Anaconda\lib\site-packages\ipykernel_launcher.py:5: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  """
E:\Anaconda\lib\site-packages\sklearn\preprocessing\data.py:617: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
E:\Anaconda\lib\site-packages\ipykernel_launcher.py:8: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  
E:\Anaconda\lib\site-packages\ipykernel_launche

In [6]:
#logostic regression

#-----EI-----

logreg=LogisticRegression(solver='lbfgs', max_iter=500)
logreg.fit(train_EI_scaled,train_data_EI_target.values.ravel())
train_pred_EI = cross_validate(logreg, train_EI_scaled, train_data_EI_target.values.ravel(), cv=10, scoring ='accuracy')
test_pred_EI = logreg.predict(test_EI_scaled)
accuracy_EI = accuracy_score(test_data_EI_target, test_pred_EI)
print("Test Accuracy Score EI:")
print(accuracy_EI)

filename = 'logreg_EI.sav'
joblib.dump(logreg, filename)

Test Accuracy Score EI:
0.82625


['logreg_EI.sav']

In [7]:
#logistic regression

#----FT----

logreg=LogisticRegression(solver='lbfgs', max_iter=500)
logreg.fit(train_FT_scaled,train_data_FT_target.values.ravel())
train_pred_FT = cross_validate(logreg, train_FT_scaled, train_data_FT_target.values.ravel(), cv=10, scoring ='accuracy')
test_pred_FT = logreg.predict(test_FT_scaled)
accuracy_FT = accuracy_score(test_data_FT_target, test_pred_FT)
print("Test Accuracy Score FT:")
print(accuracy_FT)

filename = 'logreg_FT.sav'
joblib.dump(logreg, filename)

Test Accuracy Score FT:
0.9516635279347144


['logreg_FT.sav']

In [8]:
#logistic regression

#----NS----

logreg=LogisticRegression(solver='lbfgs', max_iter=500)
logreg.fit(train_NS_scaled,train_data_NS_target.values.ravel())
train_pred_NS = cross_validate(logreg, train_NS_scaled, train_data_NS_target.values.ravel(), cv=10, scoring ='accuracy')
test_pred_NS = logreg.predict(test_NS_scaled)
accuracy_NS = accuracy_score(test_data_NS_target, test_pred_NS)
print("Test Accuracy Score NS:")
print(accuracy_NS)

filename = 'logreg_NS.sav'
joblib.dump(logreg, filename)

Test Accuracy Score NS:
0.7432150313152401


['logreg_NS.sav']

In [9]:
#logistic regression

#----PJ----

logreg=LogisticRegression(solver='lbfgs', max_iter=500)
logreg.fit(train_PJ_scaled,train_data_PJ_target.values.ravel())
train_pred_PJ = cross_validate(logreg, train_PJ_scaled, train_data_PJ_target.values.ravel(), cv=10, scoring ='accuracy')
test_pred_PJ = logreg.predict(test_PJ_scaled)
accuracy_PJ = accuracy_score(test_data_PJ_target, test_pred_PJ)
print("Test Accuracy Score PJ:")
print(accuracy_PJ)

filename = 'logreg_PJ.sav'
joblib.dump(logreg, filename)

Test Accuracy Score PJ:
0.9090247452692868


['logreg_PJ.sav']

In [10]:
#random forest

rft1 = RandomForestClassifier()
rft1.fit(train_EI_scaled,train_data_EI_target.values.ravel())
train_pred_EI = cross_validate(rft1, train_EI_scaled, train_data_EI_target.values.ravel(), cv=10, scoring ='accuracy')
test_pred_EI = rft1.predict(test_EI_scaled)
accuracy_EI = accuracy_score(test_data_EI_target, test_pred_EI)
print("Test Accuracy Score EI:")
print(accuracy_EI)

filename = 'randomforest_EI.sav'
joblib.dump(rft1, filename)

D:\Claudia\TensorFlow\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Test Accuracy Score EI:
0.825


['randomforest_EI.sav']

In [11]:
rft1 = RandomForestClassifier()
rft1.fit(train_FT_scaled,train_data_FT_target.values.ravel())
train_pred_FT = cross_validate(rft1, train_FT_scaled, train_data_FT_target.values.ravel(), cv=10, scoring ='accuracy')
test_pred_FT = rft1.predict(test_FT_scaled)
accuracy_FT = accuracy_score(test_data_FT_target, test_pred_FT)
print("Random forest Test Accuracy Score FT:")
print(accuracy_FT)

filename = 'randomforest_FT.sav'
joblib.dump(rft1, filename)

D:\Claudia\TensorFlow\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Random forest Test Accuracy Score FT:
0.8832391713747646


['randomforest_FT.sav']

In [12]:
rft1 = RandomForestClassifier()
rft1.fit(train_NS_scaled,train_data_NS_target.values.ravel())
train_pred_NS = cross_validate(rft1, train_NS_scaled, train_data_NS_target.values.ravel(), cv=10, scoring ='accuracy')
test_pred_NS = rft1.predict(test_NS_scaled)
accuracy_NS = accuracy_score(test_data_NS_target, test_pred_NS)
print("Random forest Test Accuracy Score NS:")
print(accuracy_NS)

filename = 'randomforest_NS.sav'
joblib.dump(rft1, filename)

D:\Claudia\TensorFlow\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Random forest Test Accuracy Score NS:
0.7807933194154488


['randomforest_NS.sav']

In [13]:
rft1 = RandomForestClassifier()
rft1.fit(train_PJ_scaled,train_data_PJ_target.values.ravel())
train_pred_PJ = cross_validate(rft1, train_PJ_scaled, train_data_PJ_target.values.ravel(), cv=10, scoring ='accuracy')
test_pred_PJ = rft1.predict(test_PJ_scaled)
accuracy_PJ = accuracy_score(test_data_PJ_target, test_pred_PJ)
print("Random forest Test Accuracy Score PJ:")
print(accuracy_PJ)

filename = 'randomforest_PJ.sav'
joblib.dump(rft1, filename)

D:\Claudia\TensorFlow\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Random forest Test Accuracy Score PJ:
0.7933042212518195


['randomforest_PJ.sav']

In [14]:
#NN
batches = 400
validation = 0.2
epochs = 250
neurons = 75
l_rate = 0.00005

def build_model(neurons, input_dim, epochs, momentum, l2reg):    
    model = Sequential()
    model.add(Dense(neurons, input_dim = input_dim, kernel_constraint=max_norm(3),
                    kernel_regularizer=regularizers.l2(l2reg),
                    activation = 'relu'))
    model.add(Dropout(0.6))
    model.add(Dense(neurons, input_dim = input_dim, kernel_constraint=max_norm(3),
                    kernel_regularizer=regularizers.l2(l2reg),
                    activation = 'relu'))    
    model.add(Dropout(0.6))
    model.add(Dense(neurons, input_dim = input_dim, kernel_constraint=max_norm(3),
                    kernel_regularizer=regularizers.l2(l2reg),
                    activation = 'relu'))
    model.add(Dropout(0.6))   
    model.add(Dense(neurons, input_dim = input_dim, kernel_constraint=max_norm(3),
                    kernel_regularizer=regularizers.l2(l2reg),
                    activation = 'relu'))
  #  model.add(Dropout(0.2))
    model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer= 'adam', metrics=["accuracy"])
    return model

In [15]:
#EI
model_EI = build_model(neurons,train_EI_scaled.shape[1],  epochs, 0.85, 0.0001)
callbacks1 = [EarlyStopping(monitor='val_loss', patience=3),
             ModelCheckpoint(filepath='best_model_t1.h5', monitor='val_loss', save_best_only=True)]
model_EI_trained = model_EI.fit(x = train_EI_scaled, y = train_data_EI_target, callbacks=callbacks1,epochs = epochs, batch_size = 64, verbose = True,  validation_split= validation)
eval_train = model_EI.evaluate(x = train_EI_scaled, y =train_data_EI_target)
eval_test = model_EI.evaluate(x = test_EI_scaled, y = test_data_EI_target)
print('training results ', eval_train )
print('testing results ', eval_test )

filename = 'NN_EI.sav'
joblib.dump(model_EI, filename)

Train on 2558 samples, validate on 640 samples
Epoch 1/250
2558/2558 [==============================] - 2s 868us/step - loss: 0.7509 - acc: 0.5102 - val_loss: 0.7290 - val_acc: 0.5469
Epoch 2/250
2558/2558 [==============================] - 1s 332us/step - loss: 0.7346 - acc: 0.5375 - val_loss: 0.7264 - val_acc: 0.5531
Epoch 3/250
2558/2558 [==============================] - 1s 335us/step - loss: 0.7285 - acc: 0.5450 - val_loss: 0.7189 - val_acc: 0.6094
Epoch 4/250
2558/2558 [==============================] - 1s 326us/step - loss: 0.7036 - acc: 0.5942 - val_loss: 0.6957 - val_acc: 0.6781
Epoch 5/250
2558/2558 [==============================] - 1s 331us/step - loss: 0.6469 - acc: 0.6603 - val_loss: 0.6434 - val_acc: 0.7000
Epoch 6/250
2558/2558 [==============================] - 1s 324us/step - loss: 0.5403 - acc: 0.7592 - val_loss: 0.6078 - val_acc: 0.7188
Epoch 7/250
2558/2558 [==============================] - 1s 326us/step - loss: 0.4541 - acc: 0.8307 - val_loss: 0.5928 - val_acc: 0

['NN_EI.sav']

In [16]:
#FT
model_FT = build_model(neurons,train_FT_scaled.shape[1],  epochs, 0.85, 0.0001)
callbacks1 = [EarlyStopping(monitor='val_loss', patience=3),
             ModelCheckpoint(filepath='best_model_t1.h5', monitor='val_loss', save_best_only=True)]
model_FT_trained = model_FT.fit(x = train_FT_scaled, y = train_data_FT_target, callbacks=callbacks1,epochs = epochs, batch_size = 64, verbose = True,  validation_split= validation)
eval_train = model_FT.evaluate(x = train_FT_scaled, y =train_data_FT_target)
eval_test = model_FT.evaluate(x = test_FT_scaled, y = test_data_FT_target)
print('FT training results ', eval_train )
print('FT testing results ', eval_test )

filename = 'NN_FT.sav'
joblib.dump(model_FT, filename)

Train on 5095 samples, validate on 1274 samples
Epoch 1/250
5095/5095 [==============================] - 2s 451us/step - loss: 0.7452 - acc: 0.5111 - val_loss: 0.7214 - val_acc: 0.6193
Epoch 2/250
5095/5095 [==============================] - 2s 349us/step - loss: 0.7088 - acc: 0.5819 - val_loss: 0.6404 - val_acc: 0.7771
Epoch 3/250
5095/5095 [==============================] - 2s 351us/step - loss: 0.5718 - acc: 0.7441 - val_loss: 0.4459 - val_acc: 0.8422
Epoch 4/250
5095/5095 [==============================] - 2s 332us/step - loss: 0.4075 - acc: 0.8553 - val_loss: 0.3897 - val_acc: 0.8658
Epoch 5/250
5095/5095 [==============================] - 2s 334us/step - loss: 0.2968 - acc: 0.9056 - val_loss: 0.3807 - val_acc: 0.8689
Epoch 6/250
5095/5095 [==============================] - 2s 332us/step - loss: 0.2188 - acc: 0.9394 - val_loss: 0.4122 - val_acc: 0.8736
Epoch 7/250
5095/5095 [==============================] - 2s 334us/step - loss: 0.1840 - acc: 0.9529 - val_loss: 0.4218 - val_acc: 

['NN_FT.sav']

In [17]:
#NS
model_NS = build_model(neurons,train_NS_scaled.shape[1],  epochs, 0.85, 0.0001)
callbacks1 = [EarlyStopping(monitor='val_loss', patience=3),
             ModelCheckpoint(filepath='best_model_t1.h5', monitor='val_loss', save_best_only=True)]
model_NS_trained = model_NS.fit(x = train_NS_scaled, y = train_data_NS_target, callbacks=callbacks1,epochs = epochs, batch_size = 64, verbose = True,  validation_split= validation)
eval_train = model_NS.evaluate(x = train_NS_scaled, y =train_data_NS_target)
eval_test = model_NS.evaluate(x = test_NS_scaled, y = test_data_NS_target)
print('NS training results ', eval_train )
print('NS testing results ', eval_test )

filename = 'NN_NS.sav'
joblib.dump(model_NS, filename)

Train on 1532 samples, validate on 383 samples
Epoch 1/250
1532/1532 [==============================] - 1s 781us/step - loss: 0.7486 - acc: 0.5163 - val_loss: 0.7342 - val_acc: 0.4883
Epoch 2/250
1532/1532 [==============================] - 0s 325us/step - loss: 0.7446 - acc: 0.5261 - val_loss: 0.7334 - val_acc: 0.4935
Epoch 3/250
1532/1532 [==============================] - 1s 327us/step - loss: 0.7458 - acc: 0.5131 - val_loss: 0.7304 - val_acc: 0.5535
Epoch 4/250
1532/1532 [==============================] - 0s 313us/step - loss: 0.7226 - acc: 0.5627 - val_loss: 0.7285 - val_acc: 0.5535
Epoch 5/250
1532/1532 [==============================] - 1s 341us/step - loss: 0.7167 - acc: 0.5646 - val_loss: 0.7227 - val_acc: 0.5953
Epoch 6/250
1532/1532 [==============================] - 1s 330us/step - loss: 0.7031 - acc: 0.5888 - val_loss: 0.7134 - val_acc: 0.6162
Epoch 7/250
1532/1532 [==============================] - 1s 327us/step - loss: 0.6542 - acc: 0.6495 - val_loss: 0.7000 - val_acc: 0

['NN_NS.sav']

In [18]:
#PJ
model_PJ = build_model(neurons,train_PJ_scaled.shape[1],  epochs, 0.85, 0.0001)
callbacks1 = [EarlyStopping(monitor='val_loss', patience=3),
             ModelCheckpoint(filepath='best_model_t1.h5', monitor='val_loss', save_best_only=True)]
model_PJ_trained = model_PJ.fit(x = train_PJ_scaled, y = train_data_PJ_target, callbacks=callbacks1,epochs = epochs, batch_size = 64, verbose = True,  validation_split= validation)
eval_train = model_PJ.evaluate(x = train_PJ_scaled, y =train_data_PJ_target)
eval_test = model_PJ.evaluate(x = test_PJ_scaled, y = test_data_PJ_target)
print('PJ training results ', eval_train )
print('PJ testing results ', eval_test )

filename = 'NN_PJ.sav'
joblib.dump(model_PJ, filename)

Train on 4395 samples, validate on 1099 samples
Epoch 1/250
4395/4395 [==============================] - 2s 500us/step - loss: 0.7534 - acc: 0.4933 - val_loss: 0.7318 - val_acc: 0.5250
Epoch 2/250
4395/4395 [==============================] - 1s 329us/step - loss: 0.7420 - acc: 0.5135 - val_loss: 0.7308 - val_acc: 0.5532
Epoch 3/250
4395/4395 [==============================] - 1s 330us/step - loss: 0.7324 - acc: 0.5395 - val_loss: 0.7260 - val_acc: 0.6187
Epoch 4/250
4395/4395 [==============================] - 1s 329us/step - loss: 0.7103 - acc: 0.5804 - val_loss: 0.6936 - val_acc: 0.6870
Epoch 5/250
4395/4395 [==============================] - 1s 329us/step - loss: 0.6507 - acc: 0.6694 - val_loss: 0.6021 - val_acc: 0.7607
Epoch 6/250
4395/4395 [==============================] - 1s 336us/step - loss: 0.5101 - acc: 0.7784 - val_loss: 0.5118 - val_acc: 0.7989
Epoch 7/250
4395/4395 [==============================] - 1s 329us/step - loss: 0.3708 - acc: 0.8733 - val_loss: 0.4874 - val_acc: 

['NN_PJ.sav']

In [ ]:
#SVM
#EI
clf = svm.SVC(gamma='scale')
clf.fit(train_EI_scaled, train_data_EI_target.values.ravel())
train_pred_EI = cross_validate(clf, train_EI_scaled, train_data_EI_target.values.ravel(), cv=10, scoring ='accuracy')
test_pred_EI = clf.predict(test_EI_scaled)
accuracy_EI = accuracy_score(test_data_EI_target, test_pred_EI)
print("Test Accuracy Score EI:")
print(accuracy_EI)

filename = 'svm_EI.sav'
joblib.dump(clf, filename)

In [ ]:
#FT
clf = svm.SVC(gamma='scale')
clf.fit(train_FT_scaled, train_data_FT_target.values.ravel())
train_pred_FT = cross_validate(clf, train_FT_scaled, train_data_FT_target.values.ravel(), cv=10, scoring ='accuracy')
test_pred_FT = clf.predict(test_FT_scaled)
accuracy_FT = accuracy_score(test_data_FT_target, test_pred_FT)
print("Test Accuracy Score FT:")
print(accuracy_FT)

filename = 'svm_FT.sav'
joblib.dump(clf, filename)

In [ ]:
#NS
clf = svm.SVC(gamma='scale')
clf.fit(train_NS_scaled, train_data_NS_target.values.ravel())
train_pred_NS = cross_validate(clf, train_NS_scaled, train_data_NS_target.values.ravel(), cv=10, scoring ='accuracy')
test_pred_NS = clf.predict(test_NS_scaled)
accuracy_NS = accuracy_score(test_data_NS_target, test_pred_NS)
print("Test Accuracy Score NS:")
print(accuracy_NS)

filename = 'svm_NS.sav'
joblib.dump(clf, filename)

In [ ]:
#PJ
clf = svm.SVC(gamma='scale')
clf.fit(train_PJ_scaled, train_data_PJ_target.values.ravel())
train_pred_PJ = cross_validate(clf, train_PJ_scaled, train_data_PJ_target.values.ravel(), cv=10, scoring ='accuracy')
test_pred_PJ = clf.predict(test_PJ_scaled)
accuracy_PJ = accuracy_score(test_data_PJ_target, test_pred_PJ)
print("Test Accuracy Score PJ:")
print(accuracy_PJ)

filename = 'svm_PJ.sav'
joblib.dump(clf, filename)

In [ ]:
#from sklearn import model_selection
#from sklearn.externals import joblib

In [20]:
print(sum(np.asfarray(train_data_EI_target[1],float)) / len(train_data_EI_target[1]))
print(sum(np.asfarray(train_data_FT_target[1],float)) / len(train_data_FT_target[1]))
print(sum(np.asfarray(train_data_NS_target[1],float)) / len(train_data_NS_target[1]))
print(sum(np.asfarray(train_data_PJ_target[1],float)) / len(train_data_PJ_target[1]))


0.49749843652282677
0.503061705134244
0.5018276762402089
0.49963596650891884


In [6]:
# load the model from disk
predict_data = pd.read_csv("../data/yelp_pre-processed_2.csv")
predict_data_r  = predict_data.drop([predict_data.columns[1],'user_id','text','text.1','posts','joined_comment'], axis = 1)


In [7]:

predict_data_scaled_EI = train_EI_scaler.transform(predict_data_r)
predict_data_scaled_NS = train_NS_scaler.transform(predict_data_r)
predict_data_scaled_FT = train_FT_scaler.transform(predict_data_r)
predict_data_scaled_PJ = train_PJ_scaler.transform(predict_data_r)


E:\Anaconda\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  
E:\Anaconda\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  This is separate from the ipykernel package so we can avoid doing imports until
E:\Anaconda\lib\site-packages\ipykernel_launcher.py:4: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  after removing the cwd from sys.path.
E:\Anaconda\lib\site-packages\ipykernel_launcher.py:5: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  """


In [8]:
predict_data.shape

(65109, 5023)

In [9]:
#EI
filename = 'logreg_EI.sav'
loaded_model_EI = joblib.load(filename)
result_EI = loaded_model_EI.predict(predict_data_scaled_EI)
#FT
filename = 'logreg_FT.sav'
loaded_model_FT = joblib.load(filename)
result_FT = loaded_model_FT.predict(predict_data_scaled_FT)
#NS
filename = 'randomforest_NS.sav'
loaded_model_NS = joblib.load(filename)
result_NS = loaded_model_NS.predict(predict_data_scaled_NS)
#PJ
filename = 'logreg_PJ.sav'
loaded_model_PJ = joblib.load(filename)
result_PJ = loaded_model_PJ.predict(predict_data_scaled_PJ)

E:\Anaconda\lib\site-packages\sklearn\base.py:251: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.20.1 when using version 0.20.0. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
E:\Anaconda\lib\site-packages\sklearn\base.py:251: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.20.1 when using version 0.20.0. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
E:\Anaconda\lib\site-packages\sklearn\base.py:251: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.20.1 when using version 0.20.0. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
E:\Anaconda\lib\site-packages\sklearn\base.py:251: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.20.1 when using version 0.20.0. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarni

In [10]:
print(result_EI)
print(result_FT)
print(result_NS)
print(result_PJ)

['0' '0' '0' ... '1' '1' '1']
['1' '1' '0' ... '1' '1' '1']
['1' '0' '0' ... '0' '0' '0']
['1' '1' '1' ... '0' '0' '0']


In [11]:
print(sum(np.asfarray(result_EI,float)) / len(result_EI))
print(sum(np.asfarray(result_FT,float)) / len(result_FT))
print(sum(np.asfarray(result_NS,float)) / len(result_NS))
print(sum(np.asfarray(result_PJ,float)) / len(result_PJ))


0.954875670030257
0.8953293707475157
0.13468184122010782
0.08665468675605523


In [12]:
predict_data.head()

,Unnamed: 0,Unnamed: 0.1,user_id,text,posts,joined_comment,max_polarity,average_polarity,min_polarity,max_subjectivity,...,youtube,youve,youyou,yr,yup,zen,zero,zodiac,zombie,zone
0,0,0,---1lKK3aKOuomHnwAkAow,I wish I had known more about this when I was ...,"['wish', 'known', 'pregnant', 'seems', 'make',...",wish known pregnant seems make sense interesti...,1.000,0.148,-1.00,1.000,...,0,0,0,0,0,0,1,0,1,0
1,1,1,--2vR0DIsmQ6WfcSzKWigw,"Wer mehrere Tage in Las Vegas ist, sollte s ni...","['wer', 'mehrere', 'tage', 'la', 'vega', 'ist'...",wer mehrere tage la vega ist sollte nicht vers...,0.600,0.013,-0.70,1.000,...,0,0,0,0,0,0,0,0,0,0
2,2,2,--3l8wysfp49Z2TLnyT0vg,"This is the best Thai restaurant in Vegas, for...","['best', 'thai', 'restaurant', 'vega', 'matter...",best thai restaurant vega matter best thai res...,0.613,0.112,-0.25,0.667,...,0,0,0,0,0,0,0,0,0,0
3,3,3,--3loIEaXcepp9OKGi_1FQ,I got the pasta carbonara and WOW!!! Amazing a...,"['got', 'pasta', 'carbonara', 'wow', 'amazing'...",got pasta carbonara wow amazing amazing amazin...,0.650,0.427,0.20,1.000,...,0,0,0,0,0,0,0,0,0,0
4,4,4,--8g9UaBe0xQ4FD0q34h_A,I was really expecting more... I had high hope...,"['really', 'expecting', 'high', 'hope', 'went'...",really expecting high hope went dessert ordere...,0.700,0.179,0.00,1.000,...,0,0,0,0,0,0,0,0,0,0


In [17]:
predict_data_n = predict_data
predict_data_n['EI'] = result_EI
predict_data_n['FT'] = result_FT
predict_data_n['NS'] = result_NS
predict_data_n['PJ'] = result_PJ

In [18]:
predict_data_r.shape

(65109, 5017)

In [19]:
predict_data_n.head(100)

,Unnamed: 0,Unnamed: 0.1,user_id,text,posts,joined_comment,max_polarity,average_polarity,min_polarity,max_subjectivity,...,yup,zen,zero,zodiac,zombie,zone,EI,FT,NS,PJ
0,0,0,---1lKK3aKOuomHnwAkAow,I wish I had known more about this when I was ...,"['wish', 'known', 'pregnant', 'seems', 'make',...",wish known pregnant seems make sense interesti...,1.000,0.148,-1.00,1.000,...,0,0,1,0,1,0,0,1,1,1
1,1,1,--2vR0DIsmQ6WfcSzKWigw,"Wer mehrere Tage in Las Vegas ist, sollte s ni...","['wer', 'mehrere', 'tage', 'la', 'vega', 'ist'...",wer mehrere tage la vega ist sollte nicht vers...,0.600,0.013,-0.70,1.000,...,0,0,0,0,0,0,0,1,0,1
2,2,2,--3l8wysfp49Z2TLnyT0vg,"This is the best Thai restaurant in Vegas, for...","['best', 'thai', 'restaurant', 'vega', 'matter...",best thai restaurant vega matter best thai res...,0.613,0.112,-0.25,0.667,...,0,0,0,0,0,0,0,0,0,1
3,3,3,--3loIEaXcepp9OKGi_1FQ,I got the pasta carbonara and WOW!!! Amazing a...,"['got', 'pasta', 'carbonara', 'wow', 'amazing'...",got pasta carbonara wow amazing amazing amazin...,0.650,0.427,0.20,1.000,...,0,0,0,0,0,0,0,0,0,1
4,4,4,--8g9UaBe0xQ4FD0q34h_A,I was really expecting more... I had high hope...,"['really', 'expecting', 'high', 'hope', 'went'...",really expecting high hope went dessert ordere...,0.700,0.179,0.00,1.000,...,0,0,0,0,0,0,0,0,1,1
5,5,5,--BumyUHiO_7YsHurb9Hkw,Food was good but service was lacking. \r\r\n\...,"['food', 'good', 'service', 'lacking', 'loved'...",food good service lacking loved artichoke neve...,1.000,0.297,-0.60,1.000,...,0,1,0,0,0,0,1,0,0,1
6,6,6,--HCoE1ghaAlcaAfshICgw,Very attentive staff. Even though it was busy ...,"['attentive', 'staff', 'even', 'though', 'busy...",attentive staff even though busy thursday afte...,1.000,0.258,-0.45,1.000,...,0,0,0,0,0,0,0,0,1,1
7,7,7,--IYQykIJoVtoae5WChZiw,It's such a great restaurant. We were able to ...,"['great', 'restaurant', 'able', 'sit', 'outsid...",great restaurant able sit outside beautiful fa...,0.700,0.189,-0.38,0.839,...,0,0,0,0,0,0,0,0,1,1
8,8,8,--Qh8yKWAvIP4V4K8ZPfHA,"I saw The Killers perform here on Sep 22, 2012...","['saw', 'killer', 'perform', 'sep', 'first', '...",saw killer perform sep first time venue expect...,1.000,0.139,-1.00,1.000,...,0,0,0,0,0,0,1,1,0,0
9,9,9,--RlSfc-QmcHFGHyX6aVjA,HATE Nothing.\r\r\n\r\r\nDISLIKE Location is...,"['hate', 'nothing', 'dislike', 'location', 'ki...",hate nothing dislike location kind far home ca...,1.000,0.016,-0.80,1.000,...,0,0,0,0,0,0,1,1,1,0


In [16]:
predict_data_n.to_csv('predicted_personality_types.csv')

In [20]:
predict_data_scaler = preprocessing.StandardScaler().fit(predict_data_r)
predict_data_scaled = predict_data_scaler.transform(predict_data_r)

E:\Anaconda\lib\site-packages\sklearn\preprocessing\data.py:617: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
E:\Anaconda\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  


In [27]:
#EI
filename = 'logreg_EI.sav'
loaded_model_EI = joblib.load(filename)
result_EI = loaded_model_EI.predict(predict_data_scaled)
#FT
filename = 'logreg_FT.sav'
loaded_model_FT = joblib.load(filename)
result_FT = loaded_model_FT.predict(predict_data_scaled)
#NS
filename = 'randomforest_NS.sav'
loaded_model_NS = joblib.load(filename)
result_NS = loaded_model_NS.predict(predict_data_scaled)
#PJ
filename = 'logreg_PJ.sav'
loaded_model_PJ = joblib.load(filename)
result_PJ = loaded_model_PJ.predict(predict_data_scaled)

E:\Anaconda\lib\site-packages\sklearn\base.py:251: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.20.1 when using version 0.20.0. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
E:\Anaconda\lib\site-packages\sklearn\base.py:251: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.20.1 when using version 0.20.0. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
E:\Anaconda\lib\site-packages\sklearn\base.py:251: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.20.1 when using version 0.20.0. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
E:\Anaconda\lib\site-packages\sklearn\base.py:251: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.20.1 when using version 0.20.0. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarni

In [28]:
print(result_EI)
print(result_FT)
print(result_NS)
print(result_PJ)

['1' '1' '0' ... '1' '1' '1']
['1' '0' '0' ... '1' '1' '1']
['1' '0' '1' ... '0' '0' '0']
['0' '1' '1' ... '0' '0' '0']


In [29]:
print(sum(np.asfarray(result_EI,float)) / len(result_EI))
print(sum(np.asfarray(result_FT,float)) / len(result_FT))
print(sum(np.asfarray(result_NS,float)) / len(result_NS))
print(sum(np.asfarray(result_PJ,float)) / len(result_PJ))

0.4921593020934126
0.05596768495906864
0.24661721113824508
0.8742723740189529


In [30]:
predict_data_n = predict_data
predict_data_n['EI'] = result_EI
predict_data_n['FT'] = result_FT
predict_data_n['NS'] = result_NS
predict_data_n['PJ'] = result_PJ
predict_data_n.to_csv('predicted_personality_types.csv')